# CSI 4142 Data Science 
## Assignment 2 - Data Cleaning

### Identification

Name: Eli Wynn<br/>
Student Number: 300248135

Name: Jack Snelgrove<br/>
Student Number: 300247435


Our datasets have been uploaded from the public repository:

- [github.com/eli-wynn/Datasets](https://github.com/eli-wynn/Datasets)

Imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### Importing Datasets

### Clean Data Checker

#### Data Type Error

A data type error occurs when the the data entered into a column doesnt match the data type assigned to that column. 


### Imputation